<a href="https://colab.research.google.com/github/vn322/Sport-University/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=2f78b868c4a806ecc8fee810f7e2d8135f3ec72417f587078c94791b46a860b8
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [4]:
# importing required libraries and initializing spark session
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
#create instance of spark class 
spark=SparkSession.builder.appName('project').getOrCreate()

spark

In [9]:
import pyspark.sql.functions as f
#importing the cleaned olympics dataset and creating a pyspark dataframe 
file_location = "/content/olympix_master_cleaned.csv"
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_location)
# Taking a glimpse of the pyspark dataframe created using show function
df.show()

+--------------------+----+-------------+----+----------+--------------------+----------+------------+-----------+-----------+
|                name| age|      country|year|Date_Given|              sports|gold_medal|silver_medal|brone_medal|total_medal|
+--------------------+----+-------------+----+----------+--------------------+----------+------------+-----------+-----------+
|      Michael Phelps|23.0|United States|2008| 8/24/2008|            Swimming|         8|           0|          0|          8|
|      Michael Phelps|19.0|United States|2004| 8/29/2004|            Swimming|         6|           0|          2|          8|
|      Michael Phelps|27.0|United States|2012|08-12-2012|            Swimming|         4|           2|          0|          6|
|    Natalie Coughlin|25.0|United States|2008| 8/24/2008|            Swimming|         1|           2|          3|          6|
|       Aleksey Nemov|24.0|       Russia|2000|10-01-2000|          Gymnastics|         2|           1|         

In [11]:
# Creating a dataframe with set of sports as sports column contains repeated values. 
df1=df.select(f.collect_set("sports"))
df1 = df1.withColumnRenamed("collect_set(sports)","Olympic_Sports/games")
df1.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Olympic_Sports/games                                                                                                                                                                                                                                                                                                                                                                                     

In [12]:
df.select(f.countDistinct('sports')).show()

+----------------------+
|count(DISTINCT sports)|
+----------------------+
|                    49|
+----------------------+



In [13]:
# Counting the frequency of each distinct sports in the dataset using groupby function
df.groupBy('sports').count().show(df.count(),False)

+-------------------------+-----+
|sports                   |count|
+-------------------------+-----+
|Gymnastics               |194  |
|Tennis                   |71   |
|Boxing                   |188  |
|Short-Track Speed Skating|96   |
|Ice Hockey               |384  |
|Rowing                   |567  |
|Judo                     |224  |
|Softball                 |134  |
|Sailing                  |210  |
|Swimming                 |487  |
|Alpine Skiing            |61   |
|Basketball               |287  |
|Handball                 |351  |
|Rhythmic Gymnastics      |84   |
|Biathlon                 |94   |
|Triathlon                |24   |
|Badminton                |91   |
|Athletics                |687  |
|Figure Skating           |54   |
|Beach Volleyball         |48   |
|Equestrian               |157  |
|Shooting                 |181  |
|Diving                   |113  |
|Skeleton                 |18   |
|Ski Jumping              |40   |
|Nordic Combined          |39   |
|Wrestling    

In [14]:
df2 = df.groupBy(["sports"]).sum("total_medal")
df2.orderBy('sports').show(df2.count(),False)

+-------------------------+----------------+
|sports                   |sum(total_medal)|
+-------------------------+----------------+
|Alpine Skiing            |90              |
|Archery                  |96              |
|Athletics                |753             |
|Badminton                |96              |
|Baseball                 |216             |
|Basketball               |287             |
|Beach Volleyball         |48              |
|Biathlon                 |138             |
|Bobsleigh                |72              |
|Boxing                   |188             |
|Canoeing                 |333             |
|Cross Country Skiing     |174             |
|Curling                  |82              |
|Cycling                  |306             |
|Diving                   |144             |
|Equestrian               |182             |
|Fencing                  |256             |
|Figure Skating           |54              |
|Football                 |407             |
|Freestyle

In [15]:
# Creating a dataframe by grouping sports yearwise and taking total medal count for that year for that sport
df3 = df.groupBy(['year',"sports"]).sum("total_medal")
df3.orderBy('year','sports').show(df3.count(),False)

+----+-------------------------+----------------+
|year|sports                   |sum(total_medal)|
+----+-------------------------+----------------+
|2000|Archery                  |24              |
|2000|Athletics                |190             |
|2000|Badminton                |24              |
|2000|Baseball                 |72              |
|2000|Basketball               |71              |
|2000|Beach Volleyball         |12              |
|2000|Boxing                   |48              |
|2000|Canoeing                 |84              |
|2000|Cycling                  |74              |
|2000|Diving                   |36              |
|2000|Equestrian               |45              |
|2000|Fencing                  |66              |
|2000|Football                 |99              |
|2000|Gymnastics               |72              |
|2000|Handball                 |90              |
|2000|Hockey                   |96              |
|2000|Judo                     |56              |


In [16]:
# using orderBy to sort results and using groupby on sports with aggregate function sum on total medals 
df4 = df.groupBy("sports").sum("total_medal")
df4.orderBy('sum(total_medal)',ascending=False).show(df4.count(),False)

+-------------------------+----------------+
|sports                   |sum(total_medal)|
+-------------------------+----------------+
|Swimming                 |765             |
|Athletics                |753             |
|Rowing                   |576             |
|Football                 |407             |
|Hockey                   |386             |
|Ice Hockey               |384             |
|Handball                 |351             |
|Canoeing                 |333             |
|Waterpolo                |306             |
|Cycling                  |306             |
|Basketball               |287             |
|Gymnastics               |282             |
|Volleyball               |278             |
|Fencing                  |256             |
|Wrestling                |245             |
|Judo                     |224             |
|Baseball                 |216             |
|Sailing                  |210             |
|Shooting                 |192             |
|Boxing   

In [17]:
# Filtering dataframe for swimming sport and agin using groupby on country with aggregate function sum on total medals.
df5 = df.filter(df.sports =="Swimming").select(["country",'sports',"total_medal"])
df5 = df5.groupBy("country").sum("total_medal")
df5.orderBy(f.desc("sum(total_medal)")).withColumnRenamed("sum(total_medal)","total_medal_Swimming").show(df5.count(),False)

+-------------------+--------------------+
|country            |total_medal_Swimming|
+-------------------+--------------------+
|United States      |267                 |
|Australia          |163                 |
|Netherlands        |46                  |
|Japan              |43                  |
|France             |39                  |
|China              |35                  |
|Germany            |32                  |
|Russia             |20                  |
|Italy              |16                  |
|Great Britain      |11                  |
|South Africa       |11                  |
|Sweden             |9                   |
|Hungary            |9                   |
|Brazil             |8                   |
|Ukraine            |7                   |
|Zimbabwe           |7                   |
|Romania            |6                   |
|Canada             |5                   |
|South Korea        |4                   |
|Poland             |3                   |
|Austria   

In [18]:
# Filtering dataframe for Skeleton sport and again using groupby on country with aggregate function sum on total medals.
df6 = df.filter(df.sports =="Skeleton").select(["country",'sports',"total_medal"])
df6 = df6.groupBy("country").sum("total_medal")
df6.orderBy(f.desc("sum(total_medal)")).withColumnRenamed("sum(total_medal)","total_medal_Skeleton").show(df6.count(),False)

+-------------+--------------------+
|country      |total_medal_Skeleton|
+-------------+--------------------+
|Canada       |4                   |
|Great Britain|3                   |
|Switzerland  |3                   |
|United States|3                   |
|Germany      |2                   |
|Russia       |1                   |
|Austria      |1                   |
|Latvia       |1                   |
+-------------+--------------------+



In [19]:
# Filtering dataframe for United States country and using groupby on year with aggregate function sum on total medals.
df7 = df.filter(df.country =="Russia").select(["country",'year',"total_medal"])
df7 = df7.groupBy("year").sum("total_medal")
df7.orderBy('year').withColumnRenamed("sum(total_medal)","total_medal_US_won").show(df7.count(),False)

+----+------------------+
|year|total_medal_US_won|
+----+------------------+
|2000|187               |
|2002|38                |
|2004|191               |
|2006|41                |
|2008|146               |
|2010|25                |
|2012|140               |
+----+------------------+



In [20]:
# Using groupby on country with aggregate function sum on total medals.
df8 = df.groupBy("country").sum("total_medal")
df8.orderBy(f.desc("sum(total_medal)")).withColumnRenamed("sum(total_medal)","total_medal_won").show(df8.count(),False)


+---------------------+---------------+
|country              |total_medal_won|
+---------------------+---------------+
|United States        |1312           |
|Russia               |768            |
|Germany              |629            |
|Australia            |609            |
|China                |530            |
|Canada               |370            |
|Italy                |331            |
|Great Britain        |322            |
|France               |318            |
|Netherlands          |318            |
|South Korea          |308            |
|Japan                |282            |
|Brazil               |218            |
|Spain                |205            |
|Norway               |192            |
|Cuba                 |188            |
|Sweden               |181            |
|Hungary              |145            |
|Ukraine              |143            |
|Argentina            |139            |
|Romania              |123            |
|Finland              |118            |


In [21]:
# Filtering out the maximum age from age column and printing name and age
df.select('name','age','sports','silver_medal','year').filter(df.age == df.agg({"age":"max"}).collect()[0][0]).show()

+----------+----+----------+------------+----+
|      name| age|    sports|silver_medal|year|
+----------+----+----------+------------+----+
|Ian Millar|61.0|Equestrian|           1|2008|
+----------+----+----------+------------+----+



In [22]:
# Filtering out the maximum age from age column and printing name and country
df.select('name','country').filter(df.age == df.agg({"age":"max"}).collect()[0][0]).show()

+----------+-------+
|      name|country|
+----------+-------+
|Ian Millar| Canada|
+----------+-------+

